In [2]:
import cv2
import numpy as np

class SB:
    def __init__(self):
        #self.img = img
        self.sobelX = None
        self.sobelY = None
        self.sb = None

    # use kernel to filter the input image
    def kernel_filter (self, kernel):
        height, width = self.img.shape
        kernel_height, kernel_width = kernel.shape

        img_pad = np.zeros(((height+2), (width+2)), np.float32)
        img_pad[1:height+1, 1:width+1] = self.img

        window = np.zeros((kernel_height, kernel_width), np.float32)
        value = np.zeros((kernel_height, kernel_width), np.float32)
        img_filter = np.zeros((height, width))

        for row in range(1, height):
            for col in range(1, width):
                window = img_pad[(row - 1): (row + 2), (col - 1): (col + 2)]
                value = window*kernel
                img_filter[row-1, col-1] = int(value.sum())

        return img_filter

    def filter(self):
        filterX = np.array([[1, 0, -1], [2, 0, -2], [1, 0, -1]])
        filterY = np.array([[1, 2, 1], [0, 0, 0], [-1, -2, -1]])
        filterG = np.array([(1, 2, 1), (2, 4, 2), (1, 2, 1)], np.float32)/16

        self.sobelX = self.kernel_filter(filterX)
        self.sobelY = self.kernel_filter(filterY)

        sb = cv2.magnitude(self.sobelX, self.sobelY)
        #(self.sobelX**2 + self.sobelY**2)**0.5
        #self.sb = cv2.GaussianBlur(sb, (5,5), 0)

    def threshold(self, th):
        img_thr = np.copy(self.sb)
        img_thr[img_thr >= th] = 0
        img_thr[img_thr != 0] = 255

        self.sb = img_thr

    def sobel(self, img, th=100):
        self.img = img
        self.filter()
        self.threshold(th)
        return self.sb


In [3]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import os
from openpyxl import load_workbook

#import all images in the data set
wb = load_workbook('../labels.xlsx')
ws = wb.active

images=[]
labels = []
count = 2
path='../Faces'
for root,dirs,files in os.walk(path):
    for image in files:
        if image.endswith((".jpg")):
            img = cv2.imread(root + "/" + image, 0)
            img = cv2.resize(img,(150,200))
            images.append(img)
            labels.append(ws.cell(row=count, column = 3).value)
            count += 1
            
print(len(images))
print(len(labels))

254
254


In [7]:
sb = SB()
sobel_img = []

for img in images[-5:]:
    print('1')
    sobel_img.append(sb.sobel(img))

1
1
1
1
1


In [ ]:
# Function used to calculate wrinkle density as per instructions
def wrinkle_density(img, threshold):
    Wa = np.sum(img >= threshold)
    Pa = img.shape[0] * img.shape[1]
    result = Wa/Pa
    return result

# Function used to calculate wrinkle depth as per instructions
def wrinkle_depth(img, threshold):
    Wa = img[img >= threshold]
    M = np.sum(Wa)
    result = M / (255*len(Wa))
    return result


# Function used to calculate average skin variance as per instructions
def avg_skin_variance(img):
    M = np.sum(img)
    Pa = img.shape[0] * img.shape[1]
    result = M / (255*Pa) 
    return result


# Function used to calculate the wrinkle features for the 5 different parts of the face: forehead, left eye, right eye, left cheek, right cheek
def wrinkle_features(img, threshold, eyes, left_eye, right_eye, mouth): 
    corner_left_eye = {}
    corner_right_eye = {}
    forehead = {}
    cheek_left = {}
    cheek_right = {}

    #Apply sobel to get wrinkled image
    sb = SB()
    wrinkled = sb.sobel(img)


    #Define the sections for each part of the face and calculate their wringle features 
    left_patch = left_eye[0] - max(left_eye[2], right_eye[2])//4
    right_patch = left_eye[0]
    top_patch = left_eye[1]
    bottom_patch = left_eye[1] + left_eye[3]
    window = wrinkled[top_patch:bottom_patch, left_patch:right_patch]
    corner_left_eye['density'] = wrinkle_density(window, threshold)
    corner_left_eye['depth'] = wrinkle_depth(window, threshold)
    corner_left_eye['variance'] = avg_skin_variance(window)


    left_patch = right_eye[0] + right_eye[2]
    right_patch = left_patch + max(left_eye[2], right_eye[2])//4
    top_patch = right_eye[1]
    bottom_patch = right_eye[1] + right_eye[3]
    window = wrinkled[top_patch:bottom_patch, left_patch:right_patch]
    corner_right_eye['density'] = wrinkle_density(window, threshold)
    corner_right_eye['depth'] = wrinkle_depth(window, threshold)
    corner_right_eye['variance'] = avg_skin_variance(window)

    left_for = mouth[0]
    right_for = mouth[0] + mouth[2]
    top_for = bottom_for - max(mouth[0]-left_eye[0], 
            right_eye[1]+right_eye[2]-mouth[0]-mouth[2])
    bottom_for = eyes - max(left_eye[3], right_eye[3]) 
    window = wrinkled[top_for:bottom_for, left_for:right_for]
    forehead['density'] = wrinkle_density(window, threshold)
    forehead['depth'] = wrinkle_depth(window, threshold)
    forehead['variance'] = avg_skin_variance(window)


    left_cheek_left = left_eye[0]
    left_cheek_right = left_cheek_left + (mouth[0] - left_eye[0])
    left_cheek_top = eyes + min(left_eye[3], right_eye[3])//4
    left_cheek_bottom = mouth[1]
    window = wrinkled[left_cheek_top:left_cheek_bottom, left_cheek_left:left_cheek_right]
    cheek_left['density'] = wrinkle_density(window, threshold)
    cheek_left['depth'] = wrinkle_depth(window, threshold)
    cheek_left['variance'] = avg_skin_variance(window)


    right_cheek_left = rb - (right_eye[0]+right_eye[2]-mouth[0]-mouth[2])
    right_cheek_right = right_eye[0] + right_eye[2]
    right_cheek_top = eyes + min(left_eye[3], right_eye[3])//4
    right_cheek_bottom = mouth[1]
    window = wrinkled[right_cheek_top:right_cheek_bottom, right_cheek_left:right_cheek_right]
    cheek_right['density'] = wrinkle_density(window, threshold)
    cheek_right['depth'] = wrinkle_depth(window, threshold)
    cheek_right['variance'] = avg_skin_variance(window)

    return corner_left_eye, corner_right_eye, forehead, cheek_left, cheek_right

In [ ]:
def geometric_feature(img, eyes, left_eye, right_eye, nose_pos, mouth_pos):
    # first geometric feature
    D_em = mouth_pos - eyes
    D_ee = (right_eye[0]+right_eye[2]//2) - (left_eye[0]+left_eye[2]//2)
    R_em = D_em / D_ee
    # second geometric feature
    D_en = nose_pos - eye_pos
    D_nm = mouth_pos - nose_pos
    R_enm = D_en / D_nm
    return R_em, R_enm

In [ ]:
def feature_extraction(img, eyes, left_eye, right_eye, nose_pos, mouth_pos, mouth, threshold):
    """ Extract wrinkle and geometric features from input image.
    
    @param  img:    input image
    @param  eye_pos:    position of eyes' center
    @param  left_eye:   bounding box of left eye in original image
    @param  right_eye:  bounding box of right eye in original image
    @param  nose_pos:   position of nose
    @param  mouth_pos:  position of mouth
    @param  mouth_area: bounding box of mouth
    @param  wrinkle_thres:  threshold to consider a pixel a wrinkle
    @return wrinkles:   list of dictionaries storing wrinkle features for
            forehead, left/right eye corner, left/right cheek
    @return R_em, R_enm:    geometric features
    """
    wrinkles = wrinkles_features(img, threshold, eyes, left_eye, right_eye, mouth)

    R_em, R_enm  = geometric_feature(img, eyes, left_eye, right_eye, nose_pos, mouth_pos)
    return wrinkles, R_em, R_enm
